In [1]:
import xarray as xr

import utils

In [2]:
agg_methods = ["min", "mean"]
dist_types = ["hausdorff", "default", "plane", "sphere"]
cv_scores = xr.concat((xr.concat((utils.load_cv_result(f"res/{dist_type}/cluster2D/{agg_method}") for agg_method in agg_methods), dim="agg_method").assign_coords({"agg_method": agg_methods}) for dist_type in dist_types), dim="dist_type").assign_coords({"dist_type": dist_types})

TPR = cv_scores.sel(metric="test_TPR")
TNR = cv_scores.sel(metric="test_TNR")
k = 0.5
M = TPR * k + TNR * (1 - k)
M = M.assign_coords(metric="test_M")
cv_scores = xr.concat([cv_scores, M], dim="metric")

cv_scores_mean = cv_scores.mean(dim="fold", skipna=True)
cv_scores_std = cv_scores.std(dim="fold", skipna=True)
dim_order = ["use_dim", "dist_type", "agg_method", "n1", "n2", "k", "metric"]

cv_scores_mean_df = cv_scores_mean.to_dataframe("value", dim_order=dim_order)["value"].unstack(level="metric").dropna()
cv_scores_std_df = cv_scores_std.to_dataframe("value", dim_order=dim_order)["value"].unstack(level="metric").dropna()

In [3]:
sort_by = "test_roc_auc"
query = {
    # "dist_type": "sphere",
    "metric": ["test_roc_auc", "test_MCC", "test_TPR", "test_TNR", "test_M"],
}
for query["use_dim"] in ["1D1D", "1D4D", "4D4D"]:
    display(query)
    display(cv_scores_mean.sel(query).to_dataframe("value")["value"].unstack(level="metric").drop_duplicates().sort_values(by=sort_by, ascending=False).head(10))


{'metric': ['test_roc_auc', 'test_MCC', 'test_TPR', 'test_TNR', 'test_M'],
 'use_dim': '1D1D'}

metric                         test_roc_auc  test_MCC  test_TPR  test_TNR  \
dist_type agg_method n1 n2 k                                                
plane     min        2  2  29      0.623424  0.196991  0.756405  0.425357   
                           30      0.622582  0.183018  0.751961  0.413571   
                           28      0.618180  0.198443  0.745817  0.438929   
                        12 29      0.617696  0.135881  0.395294  0.745000   
default   min        2  3  30      0.617248  0.151804  0.809020  0.336071   
plane     min        2  15 29      0.617092  0.120660  0.304522  0.809402   
                        7  20      0.614872  0.119216  0.587255  0.539643   
                        12 30      0.614423  0.128174  0.390850  0.742143   
default   min        2  3  29      0.614328  0.132339  0.808954  0.317143   
plane     min        2  2  27      0.614190  0.186726  0.742484  0.430714   

metric                           test_M  
dist_type agg_method n1 n2 k             
plane     min        2  2  29  0.590881  
                           30  0.582766  
                           28  0.592373  
                        12 29  0.570147  
default   min        2  3  30  0.572546  
plane     min        2  15 29  0.556962  
                        7  20  0.563449  
                        12 30  0.566496  
default   min        2  3  29  0.563049  
plane     min        2  2  27  0.586599

{'metric': ['test_roc_auc', 'test_MCC', 'test_TPR', 'test_TNR', 'test_M'],
 'use_dim': '1D4D'}

metric                         test_roc_auc  test_MCC  test_TPR  test_TNR  \
dist_type agg_method n1 n2 k                                                
plane     min        2  15 29      0.650753  0.164179  0.326144  0.832500   
                        14 29      0.644800  0.181103  0.359020  0.818929   
                        15 30      0.643449  0.161480  0.328131  0.826166   
                        14 30      0.641929  0.181451  0.361678  0.814869   
                        15 28      0.641609  0.137976  0.314575  0.819286   
                           27      0.637770  0.145507  0.325664  0.813047   
                        14 28      0.636806  0.149368  0.346144  0.802857   
default   min        2  14 29      0.628905  0.158301  0.306318  0.834524   
plane     min        2  14 27      0.628330  0.148503  0.356543  0.790816   
default   min        1  14 29      0.628263  0.166032  0.262611  0.879579   

metric                           test_M  
dist_type agg_method n1 n2 k             
plane     min        2  15 29  0.579322  
                        14 29  0.588974  
                        15 30  0.577149  
                        14 30  0.588273  
                        15 28  0.566930  
                           27  0.569355  
                        14 28  0.574500  
default   min        2  14 29  0.570421  
plane     min        2  14 27  0.573679  
default   min        1  14 29  0.571095

{'metric': ['test_roc_auc', 'test_MCC', 'test_TPR', 'test_TNR', 'test_M'],
 'use_dim': '4D4D'}

metric                         test_roc_auc  test_MCC  test_TPR  test_TNR  \
dist_type agg_method n1 n2 k                                                
default   min        4  1  5       0.643671  0.198677  0.762222  0.429286   
                     11 1  4       0.639028  0.188362  0.938039  0.165000   
sphere    min        6  9  19      0.637778  0.208218  0.860980  0.297857   
default   min        10 1  4       0.636166  0.172906  0.926536  0.168214   
sphere    min        7  9  19      0.636005  0.176695  0.865359  0.266071   
                        10 19      0.634908  0.177014  0.857320  0.276429   
default   mean       1  1  5       0.634442  0.196400  0.650000  0.555357   
sphere    min        6  10 19      0.634328  0.197015  0.844641  0.316429   
                        9  18      0.633655  0.214424  0.860915  0.306071   
                        8  19      0.632976  0.200259  0.870458  0.280714   

metric                           test_M  
dist_type agg_method n1 n2 k             
default   min        4  1  5   0.595754  
                     11 1  4   0.551520  
sphere    min        6  9  19  0.579419  
default   min        10 1  4   0.547375  
sphere    min        7  9  19  0.565715  
                        10 19  0.566874  
default   mean       1  1  5   0.602679  
sphere    min        6  10 19  0.580535  
                        9  18  0.583493  
                        8  19  0.575586

In [4]:
keys = [
    # 1D1D case
    ("1D1D", "plane", "min", 2, 2, 28),
    # 1D4D case
    ("1D4D", "plane", "min", 2, 5, 10),
    # 4D4D case
    ("4D4D", "default", "min", 4, 1, 5),
]

metrics = query["metric"]
summary_mean = cv_scores_mean_df.loc[keys, metrics]  # type: ignore
summary_std = cv_scores_std_df.loc[keys, metrics]  # type: ignore
summary_combined = summary_mean.map("{:.3f}".format) + " ± " + summary_std.map("{:.3f}".format)
summary_combined

,,,,,metric,test_roc_auc,test_MCC,test_TPR,test_TNR,test_M
use_dim,dist_type,agg_method,n1,n2,k,,,,,
1D1D,plane,min,2,2,28,0.618 ± 0.113,0.198 ± 0.199,0.746 ± 0.139,0.439 ± 0.180,0.592 ± 0.098
1D4D,plane,min,2,5,10,0.507 ± 0.109,0.023 ± 0.199,0.623 ± 0.142,0.398 ± 0.190,0.510 ± 0.102
4D4D,default,min,4,1,5,0.644 ± 0.118,0.199 ± 0.209,0.762 ± 0.118,0.429 ± 0.190,0.596 ± 0.101
